In [1]:
%load_ext autoreload
%autoreload 2

#  Sliceguard – Find critical data segments in your data (fast)
## Unstructured Data Walkthrough

🕹️ **[Interactive Demo](https://huggingface.co/spaces/renumics/sliceguard-unstructured-data)**

Sliceguard is a python library for quickly finding **critical data slices** like outliers, errors, or biases. It works on **structured** and **unstructured** data.

This notebook showcases especially the **unstructured** data case, so if you have tabular data instead have a look at **[guide for structured data](./examples/quickstart_structured_data.ipynb)** instead

It is interesting for you if you want to do the following:
1. Find **performance issues** of your machine learning model.
2. Find **anomalies and inconsistencies** in your data.
3. Quickly **explore** your data using an interactive report to generate **insights**.

To run this notebook install and import sliceguard:

In [ ]:
!pip install sliceguard[all]

In [2]:
from sliceguard import SliceGuard
from sliceguard.data import from_huggingface
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

Now download the demo dataset with our utility function:

In [3]:
df = from_huggingface("Matthijs/snacks")

You now have the following dataframe containing an image column with a path to the raw image on the harddrive, a label and a split marker.

In [4]:
df

,image,label,split
0,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
1,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
2,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
3,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
4,/home/daniel/.cache/huggingface/datasets/downl...,apple,train
...,...,...,...
950,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
951,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
952,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation
953,/home/daniel/.cache/huggingface/datasets/downl...,watermelon,validation


## Check for data slices that are particulary different (Outliers/Errors in the data)
Here sliceguard will train an **outlier detection** model to highlight data points that are especially different from the rest. Note that we here run the outlier detection only on the class popcorn. You will easily see there is a bunch of weird cases where the popcorn is a popcorn package or people are the most salient part of the image.

You can use the **report feature** that uses [Renumics Spotlight](https://github.com/Renumics/spotlight) for visualization to dig into the reasons why a cluster is considered an outlier.

In [ ]:
sg = SliceGuard()
issues = sg.find_issues(df[df["label"] == "popcorn"], features=["image"])

In [ ]:
_ = sg.report()

## Check for data slices where models are prone to fail (hard samples, inconsistencies)
Here sliceguard will **train a classification model** and check for data slices where the accuracy score is particulary bad. You will realize that this will show you a whole bunch of **ambiguous images** in this dataset, potentially containing both classes. There are also **mislabeled** examples and images that are just **challenging** for the model.

In [ ]:
sg = SliceGuard()
issues = sg.find_issues(df, features=["image"], y="label", metric=accuracy_score, automl_task="classification")

In [ ]:
_ = sg.report()

In [ ]:
image_embeddings = sg.embeddings["image"]

## Check for weaknesses of your own model (...and hard samples + inconsistencies)
This shows how to pass your **own model predictions** into sliceguard to find slices that are performing badly according to a supplied metric function. This allows you to uncover **inconsistencies** and samples that are **hard to learn** in no time!

In [ ]:
# Train the model and predict on the same data (of course in practice you will want to split your data!!!)
# This is only for showing the principle
clf = SVC()
clf.fit(image_embeddings, df["label"])
df["predictions"] = clf.predict(image_embeddings)

In [ ]:
# Pass the predictions to sliceguard and uncover hard samples and inconsistencies.
sg = SliceGuard()
issues = sg.find_issues(df, features=["image"], y="label", y_pred="predictions", metric=accuracy_score, precomputed_embeddings={"image": image_embeddings})

In [ ]:
_ = sg.report()